# Text

In [1]:
from transformers import pipeline
#classifier = pipeline('sentiment-analysis')  #distilledbert-base-uncased-finetuned-sst-2-english

classifier = pipeline('sentiment-analysis', model = 'nlptown/bert-base-multilingual-uncased-sentiment')   #for different model use cases from model hub of huggingface


In [2]:
classifier("We love the software developed by you ")

[{'label': '5 stars', 'score': 0.7523802518844604}]

# Video


In [3]:
import cv2
import numpy as np
from keras.models import model_from_json
emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}

# load json and create model
json_file = open('emotion_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
emotion_model = model_from_json(loaded_model_json)

In [4]:
emotion_model.load_weights("emotion_model.h5")
#print("Loaded model from disk")

In [5]:
cap = cv2.VideoCapture(0)
while True:
    # Find haar cascade to draw bounding box around face
    ret, frame = cap.read()
    frame = cv2.resize(frame, (1280, 720))
    if not ret:
        break
    face_detector = cv2.CascadeClassifier('haarcascades/haarcascade_frontalface_default.xml')
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # detect faces available on camera
    num_faces = face_detector.detectMultiScale(gray_frame, scaleFactor=1.3, minNeighbors=5)

    # take each face available on the camera and Preprocess it
    for (x, y, w, h) in num_faces:
        cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (0, 255, 0), 4)
        roi_gray_frame = gray_frame[y:y + h, x:x + w]
        cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray_frame, (48, 48)), -1), 0)

        # predict the emotions
        emotion_prediction = emotion_model.predict(cropped_img)
        maxindex = int(np.argmax(emotion_prediction))
        cv2.putText(frame, emotion_dict[maxindex], (x+5, y-20), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

    cv2.imshow('Emotion Detection', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 302ms/step


KeyboardInterrupt: 

# Audio

In [6]:
import pickle

In [7]:
filename = 'mlp_aud.sav'
loaded_model = pickle.load(open(filename, 'rb'))

In [10]:
def extract_feature(file_name):
    
    X, sample_rate = librosa.load(os.path.join(file_name), res_type='kaiser_fast')
    result = np.array([])
    
    stft = np.abs(librosa.stft(X))
    chromas = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T, axis=0)
    result = np.hstack((result, chromas))

    mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
    result=np.hstack((result, mfccs))

    mels = np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate, n_mels=128).T, axis=0)
    result = np.hstack((result, mels))
        
    return result
    

In [ ]:
import sounddevice as sd
from scipy.io.wavfile import write
import librosa
import os
import pandas as pd
fs = 44100  # Sample rate
seconds = 3  # Duration of recording

while(True):
    myrecording = sd.rec(int(seconds * fs), samplerate=fs, channels=2)
    sd.wait()  # Wait until recording is finished
    write('output.wav', fs, myrecording)  # Save as WAV file 
    
    
    test = extract_feature('output.wav')
    test_pd = pd.DataFrame(test)
    test = extract_feature('output.wav')
    ##test.reshape(-1,1)
    test_pd = pd.DataFrame(test)
    test_t = test_pd.T
    test_t
    
    
    ans = loaded_model.predict(test_t)
    print(ans)
    print()

['happy']

['disgust']

['disgust']

['disgust']

